In [1]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
import dalmatian
from google.cloud import storage
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

import numpy as np

from plotly.subplots import make_subplots
from fludarabine import model_helper 
from fludarabine.helper import *
from fludarabine.plotly_helper import *

In [2]:
# Input patient information
patient_id = 'GCLL-0084'
wbc_file = '../../../data/gcll/CLL8 CLL cell count 05222025 UPDATED.csv'
treatment_file = '../../../data/gcll/GCLL_treatment.txt'
UMI_start = 1
UMI_end = 7 
treatment_start = 0


In [3]:
clusters=[1, 2, 3, 4,5,6]

In [4]:
# Manually verify the sample list order is correct

In [5]:
sample_list = ['GCLL-0084-T-01','RP-1895_PDO-31926_SM-NBY6I_GCLL-0084-T-03_v1_Custom_Selection_OnPrem',
       'RP-1895_PDO-31926_SM-NBY4W_GCLL-0084-T-04_v1_Custom_Selection_OnPrem',
       'RP-1895_PDO-31926_SM-NBY4V_GCLL-0084-T-06_v1_Custom_Selection_OnPrem','RP-1895_PDO-31926_SM-NBY61_GCLL-0084-T-07_v1_Custom_Selection_OnPrem' ,'RP-1895_PDO-31926_SM-NBY67_GCLL-0084-T-08_v1_Custom_Selection_OnPrem', 'RP-1895_PDO-31926_SM-NBY5Z_GCLL-0084-T-09_v1_Custom_Selection_OnPrem', 'GCLL-0084-T-02']

In [6]:
len(sample_list)

8

In [7]:
# Output file

In [8]:
outputfile_name = "CLL_subplot_combined_0084_wbc_updated.html"

Update patient specific information in the below cell 

In [9]:
end_treatment_df = pd.read_csv("../../../data/gcll/CLL8_end_treatment.csv")

In [10]:
# Get the treatment duration 

treatment_end = end_treatment_df[end_treatment_df.Patient == patient_id].Treatment_duration.iloc[0]

In [11]:
treatment_end

155

In [12]:

wbc_df, treatment_df, cluster_CCF_df, abundance_df, mcmc_df, tree_df = load_data(patient_id, wbc_file, treatment_file)

# Filter data for the specific patient
wbc_df_patient, times_sample, CLL_count_sample,wbc_count, all_times = filter_patient_data(wbc_df, patient_id)

# Plot CLL count over time and save as HTML
CLL_plot_html = plot_CLL_count(patient_id, times_sample, CLL_count_sample, UMI_start, UMI_end, treatment_start, treatment_end, )
    




In [13]:
times_sample

[-5, 378, 510, 797, 888, 981, 1079, 1340]

In [14]:
wbc_count

[174.2, 3.2, 3.8, 4.4, 6.3, 6.6, 6.2, 7.3, 9.6, 70.4]

In [15]:
wbc_table_html = plot_metadata_table(wbc_df_patient, patient_id )

In [16]:
 plot_tree_html = plot_tree_plotly(tree_df, 0)

In [17]:
 plot_ccf_html = plot_ccf(cluster_CCF_df, times_sample, treatment_df,)

In [40]:
branch_annotations = {
    1: "ITIH2_p.I216M, del(13q14), trisomy 12",
    2: "del(11q)", 
    
}

In [41]:
ccf_tree_html = plot_ccf_tree_combined(
    tree_df=tree_df,
    tree_selected=0,
    ccf_df=cluster_CCF_df,
    times_sample=times_sample,
    treatment_df=treatment_df,
    branch_annotations=branch_annotations

)

ITIH2_p.I216M<br>del(13q14)<br>trisomy 12


In [19]:
cluster_list, cluster_abundance = model_helper.get_abundance(abundance_df, mcmc_df, sample_list)
subclone_sample, log_subclone_sample = model_helper.calc_subclone(CLL_count_sample, cluster_abundance, cluster_list)
all_abundance = model_helper.get_all_abundance(cluster_list, mcmc_df, sample_list, times_sample, )

In [20]:
#  Add noise to all abundance

import random 

random.seed(42)
def add_uniform_noise_and_normalize(lst, low=0, high=0.01):
    
    # Add uniform noise
    noisy_lst = [x + random.uniform(low, high) for x in lst]

    
    # Normalize the list
    total_sum = sum(noisy_lst)
    
    normalized_lst = [x / total_sum for x in noisy_lst]

    return normalized_lst


subclone_cluster_iter = {}
# Iterate through each cluster
for cluster, iterations in all_abundance.items():
   
    # Iterate through each iteration in the cluster
    for iteration, value in iterations.items():
        # If the iteration is not in the new dict, add it
        if iteration not in subclone_cluster_iter:
            subclone_cluster_iter[iteration] = {}
        # Add the cluster and its value to this iteration
        subclone_cluster_iter[iteration][cluster] = value

        
        
noise_added_iter = {}

for iteration, abundances in subclone_cluster_iter.items():
    
    transposed_data = {i: list(values) for i, values in enumerate(zip(*abundances.values()), start=1)}
    
    noise_added_iter[iteration] = {}
    for time_point, abundance in transposed_data.items():
        
#         print(time_point, abundance)
        noise_abundance = add_uniform_noise_and_normalize(abundance)
    
#         print(noise_abundance)

        noise_added_iter[iteration][time_point] = add_uniform_noise_and_normalize(abundance)

    
noise_added_all_abundance = {}
for iteration, abundances in noise_added_iter.items():
    original_format = {i + 1: list(values) for i, values in enumerate(zip(*abundances.values()))}
    noise_added_all_abundance[iteration] = original_format
    
    
noise_added_original_all_abundances = {}

for iteration, abundances in noise_added_all_abundance.items():
    
    for cluster, abundance_value in abundances.items():
        if cluster not in noise_added_original_all_abundances:
            noise_added_original_all_abundances[cluster] = {}
        noise_added_original_all_abundances[cluster][iteration] = abundance_value
            

In [21]:
subclone_sample_mcmc_with_uniform_noise, log_subclone_sample_mcmc_with_uniform_noise = model_helper.calc_subclone(CLL_count_sample, noise_added_original_all_abundances, cluster_list, input_type = "mcmc")

In [22]:
times_aft_tx = [x for x in all_times if x > treatment_end]
times_aft_tx.insert(0, treatment_end)

# Set the extrapolate time after treatment
extrapolate_start_idx = 1

In [23]:
times_sample

[-5, 378, 510, 797, 888, 981, 1079, 1340]

In [24]:
times_aft_tx

[155, 226, 378, 510, 596, 797, 888, 981, 1079, 1340]

In [25]:
# Generate Plotly plot
subclone_plot_html = plot_subclones(cluster_list, times_sample, CLL_count_sample, log_subclone_sample, extrapolate_start_idx, times_aft_tx, treatment_df, treatment_end)


In [26]:
linear_model_mcmc_html = plot_linear_model_mcmc(cluster_list,times_sample,  CLL_count_sample, log_subclone_sample_mcmc_with_uniform_noise,extrapolate_start_idx,times_aft_tx,treatment_df, treatment_end)  

In [27]:
# Convert the column "CLL count estm" to float 
wbc_df_patient['CLL count estm'] = wbc_df_patient['CLL count estm'].astype(float) 
times_sliced_aft = [int(i) for i in wbc_df_patient[wbc_df_patient['CLL count estm'] > 0]['Time_since_start_tx'].values  if int(i) > 0]

/var/folders/x6/7zg8fddx4dlgx2lv4sv34_g40000gp/T/ipykernel_77754/981427378.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
times_aft_tx

[155, 226, 378, 510, 596, 797, 888, 981, 1079, 1340]

In [29]:
# Needs to manually check times_sliced_aft

In [30]:
times_sliced_aft

[226, 378, 510, 596, 797, 888, 981, 1079, 1340]

In [31]:
len(times_sliced_aft)

9

In [32]:
index_samples_model = slice(1,8)

# index_samples_model is the index of the samples after treatment. For instance, slice(1,7) means sample 2, 3, 4, 5,6, 7

X, y = create_inputs(times_sliced_aft, log_subclone_sample_mcmc_with_uniform_noise,2, index_samples_model, times_sample)



In [33]:
wbc_aft_tx = wbc_df_patient[wbc_df_patient.Time_since_start_tx > treatment_end]['CLL count estm'].to_list()
wbc_model = [i for i in wbc_aft_tx if i > 0]

In [34]:
wbc_model

[0.0068046,
 0.02347396,
 0.02831299,
 0.14321521,
 0.46336752,
 0.86777623,
 1.39366541,
 2.74048454,
 68.992]

In [35]:
len(wbc_model)

9

In [36]:
n_clusters = len(cluster_list)
logsumexp_points = np.log(wbc_model)
model = MultiClusterLinearRegression(n_clusters, X, y)
model.fit(logsumexp_points)

In [37]:
# Generate Plotly plot
subclone_plot_new_model_html = plot_subclones_new_model(cluster_list, times_sample, wbc_model, log_subclone_sample, extrapolate_start_idx, times_aft_tx, times_sliced_aft, treatment_df,treatment_end, model )


In [38]:
plot_mcmc_new_model_html = plot_mcmc_model(clusters, index_samples_model, times_aft_tx, times_sample, times_sliced_aft, sample_list, wbc_model, log_subclone_sample_mcmc_with_uniform_noise, treatment_df, treatment_end)


In [42]:
create_html_file([wbc_table_html, plot_ccf_html, ccf_tree_html, CLL_plot_html, subclone_plot_html,linear_model_mcmc_html,subclone_plot_new_model_html, plot_mcmc_new_model_html], output_file=outputfile_name)

HTML file saved as CLL_subplot_combined_0084_wbc_updated.html
